In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from matplotlib import cm
import seaborn as sns

plt.rcParams['text.usetex'] = True
cm2 = cm.get_cmap('coolwarm')
colors = sns.color_palette("Paired")

def plot_scaling():
    X, y = make_blobs(n_samples=100, centers=2, random_state=42, cluster_std=1)
    X += 10

    fig, axes = plt.subplots(1, 3, figsize=(12, 4.5), gridspec_kw={"width_ratios": [1, 1, 1]})

    titles = ["Originale Daten", "StandardScaler", "MinMaxScaler"]
    scalers = [None, StandardScaler(), MinMaxScaler()]

    for ax, title, scaler in zip(axes, titles, scalers):
        if scaler is None:
            X_ = X
        else:
            X_ = scaler.fit_transform(X)

        ax.scatter(X_[:, 0], X_[:, 1], c=y, cmap=cm2, s=20)
        ax.set_title(title)

        # Bereich setzen
        if scaler is None:
            ax.set_xlim(0, X[:, 0].max() + 1)
            ax.set_ylim(7.5, X[:, 1].max() + 1)
        else:
            ax.set_xlim(-2.5, 2.5)
            ax.set_ylim(-2.5, 2.5)

        # Linien: μ, ±σ, min/max
        for dim in [0, 1]:
            mean = X_[:, dim].mean()
            std = X_[:, dim].std()
            min_, max_ = X_[:, dim].min(), X_[:, dim].max()

            if dim == 0:
                ax.axvline(mean, color='gray', linestyle='--', label=r"$\mu$")
                ax.axvline(mean + std, color=colors[5], linestyle=':', label=r"$\mu + \sigma$")
                ax.axvline(mean - std, color=colors[4], linestyle=':', label=r"$\mu - \sigma$")
                ax.axvline(min_, color=colors[0], linestyle='-', alpha=0.2, label="Min")
                ax.axvline(max_, color=colors[1], linestyle='-', alpha=0.2, label="Max")
            else:
                ax.axhline(mean, color='gray', linestyle='--')
                ax.axhline(mean + std, color=colors[5], linestyle=':')
                ax.axhline(mean - std, color=colors[4], linestyle=':')
                ax.axhline(min_, color=colors[0], linestyle='-', alpha=0.2)
                ax.axhline(max_, color=colors[1], linestyle='-', alpha=0.2)

        # Achsenstyling
        ax.spines['top'].set_color('none')
        ax.spines['right'].set_color('none')
        ax.xaxis.set_ticks_position('bottom')
        ax.yaxis.set_ticks_position('left')

    # Legend nur bei Original
    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc="lower center", bbox_to_anchor=(0.5, 1.02), ncol=5)

    plt.tight_layout()
    plt.savefig("plots/example_scaler.pdf", bbox_inches="tight")
    plt.show()

plot_scaling()


In [ ]:
# Imports und Setup nach Reset erneut ausführen
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, savgol_filter
import seaborn as sns
plt.rcParams['text.usetex'] = True

# Beispiel-Daten
x = np.linspace(0, 6 * np.pi, 1000)
colors = sns.color_palette()
y = np.sin(x) + 0.5 * np.sin(3 * x) + 0.5 * np.random.normal(size=x.shape) + 2

# Savitzky-Golay-Filter anwenden
y_filtered = savgol_filter(y, window_length=20, polyorder=1)
mu = np.mean(y_filtered)
sigma = np.std(y_filtered)
# y_filtered = (y_filtered - mu) / sigma

# Peaks im Originalsignal
peaks, properties = find_peaks(y, prominence=1.00, width=5)
prominence = properties["prominences"]

# Peaks im geglätteten Signal
peaks_savgol, properties_savgol = find_peaks(y_filtered, prominence=0.4, width=3)
prominence_savgol = properties_savgol["prominences"]

# Plot
fig, axs = plt.subplots(2, 1, figsize=(10, 7), sharex=True)

# Originalsignal mit Peaks
axs[0].plot(y, label='Original Signal (OS)', color=colors[0])
axs[0].plot(peaks, y[peaks], 'x', label='Peaks (OS)', color=colors[4])
axs[0].vlines(x=peaks, ymin=y[peaks] - properties["prominences"], ymax=y[peaks], color=colors[3], linestyle='--', label="Prominence (OS)")
axs[0].hlines(y=properties["width_heights"], xmin=properties["left_ips"],
              xmax=properties["right_ips"], color=colors[4], linestyle='--', label="Breite des Peaks (OS)")
leg = axs[0].legend(
    loc="lower center",
    bbox_to_anchor=(0.5, 1.02),
    borderaxespad=0,
    ncol=4,
    frameon=False
)
for line in leg.get_lines():
    line.set_linewidth(5)

# Geglättetes Signal mit Peaks
axs[1].plot(y_filtered, label='Savitzky-Golay-Filter (SG)', color=colors[0])
axs[1].plot(peaks_savgol, y_filtered[peaks_savgol], 'x', label='Peaks (SG)', color=colors[4])
axs[1].vlines(x=peaks_savgol, ymin=y_filtered[peaks_savgol] - properties_savgol["prominences"],
              ymax=y_filtered[peaks_savgol], color=colors[3], linestyle='--', label="Prominence (SG)")
axs[1].hlines(y=properties_savgol["width_heights"], xmin=properties_savgol["left_ips"],
              xmax=properties_savgol["right_ips"], color=colors[4], linestyle='--', label="Breite des Peaks (SG)")
leg = axs[1].legend(
    loc="lower center",
    bbox_to_anchor=(0.5, 1.02),
    borderaxespad=0,
    ncol=4,
    frameon=False
)
for line in leg.get_lines():
    line.set_linewidth(5)

axs[0].spines['top'].set_color('none')
axs[0].spines['right'].set_color('none')
axs[1].spines['top'].set_color('none')
axs[1].spines['right'].set_color('none')

# axs[0].xaxis.set_ticks_position('bottom')
axs[0].yaxis.set_ticks_position('left')
axs[1].xaxis.set_ticks_position('bottom')
axs[1].yaxis.set_ticks_position('left')


# ax.spines['right'].set_color('none')
# ax.xaxis.set_ticks_position('bottom')
# ax.yaxis.set_ticks_position('left')
axs[0].grid(False)
axs[1].grid(False)
plt.tight_layout()
plt.savefig("plots/prominence_example.pdf")
plt.show()


In [ ]:
properties


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors

# --- Wavelength to RGB function unchanged ---
def wavelength_to_rgb(wavelength, gamma=0.8):
    wavelength = float(wavelength)
    if wavelength >= 380 and wavelength <= 750:
        A = 1.
    else:
        A = 0.5
    if wavelength < 380:
        wavelength = 380.
    if wavelength > 750:
        wavelength = 750.
    if wavelength >= 380 and wavelength <= 440:
        attenuation = 0.3 + 0.7 * (wavelength - 380) / (440 - 380)
        R = ((-(wavelength - 440) / (440 - 380)) * attenuation) ** gamma
        G = 0.0
        B = (1.0 * attenuation) ** gamma
    elif wavelength >= 440 and wavelength <= 490:
        R = 0.0
        G = ((wavelength - 440) / (490 - 440)) ** gamma
        B = 1.0
    elif wavelength >= 490 and wavelength <= 510:
        R = 0.0
        G = 1.0
        B = (-(wavelength - 510) / (510 - 490)) ** gamma
    elif wavelength >= 510 and wavelength <= 580:
        R = ((wavelength - 510) / (580 - 510)) ** gamma
        G = 1.0
        B = 0.0
    elif wavelength >= 580 and wavelength <= 645:
        R = 1.0
        G = (-(wavelength - 645) / (645 - 580)) ** gamma
        B = 0.0
    elif wavelength >= 645 and wavelength <= 750:
        attenuation = 0.3 + 0.7 * (750 - wavelength) / (750 - 645)
        R = (1.0 * attenuation) ** gamma
        G = 0.0
        B = 0.0
    else:
        R = 0.0
        G = 0.0
        B = 0.0
    return (R, G, B, A)

# --- Spectrum colormap setup ---
clim = (350, 780)
norm = plt.Normalize(*clim)
wl = np.arange(clim[0], clim[1] + 1, 2)
colorlist = list(zip(norm(wl), [wavelength_to_rgb(w) for w in wl]))
spectralmap = matplotlib.colors.LinearSegmentedColormap.from_list("spectrum", colorlist)

# --- Generate rectangular spectrum image ---
wavelengths = np.linspace(350, 780, 1000)
y = np.linspace(0, 1, 10)  # just a thin rectangle
X, Y = np.meshgrid(wavelengths, y)
extent = (np.min(wavelengths), np.max(wavelengths), np.min(y), np.max(y))

# --- Plot ---
fig, ax = plt.subplots(figsize=(10, 4), tight_layout=True)
ax.imshow(X, clim=clim, extent=extent, cmap=spectralmap, aspect='auto')

# Remove all borders, ticks, and labels
ax.set_axis_off()

# Save or show
plt.savefig("plots/clean_spectrum_bar.svg", bbox_inches='tight', pad_inches=0)
plt.show()
